These are dependencies for Python 3.6 should be similar to Python 2.7.

In [1]:
import math
import warnings
import itertools
import osmnx as ox
import numpy as np
import networkx as nx

Ignoring unwanted warnings.

In [2]:
warnings.filterwarnings('ignore')

I am using the city of Sao Carlos as an example, change as you please.

In [3]:
oxg = ox.graph_from_place('São Carlos, Sao Paulo, Brazil')

Number of Vertices and Edges in the MultiDiGraph.

In [4]:
nx.number_of_nodes(oxg), nx.number_of_edges(oxg)

(9880, 25428)

Direction is not important in this case, so let's use a simple Graph.

In [5]:
oxg = nx.Graph(oxg) # MultiDiGraph (input) ~> Graph (output)

Removing self loops from the graph.

In [6]:
oxg.remove_edges_from(oxg.selfloop_edges())

Number of Vertices and Edges after converting MultiDiGraph to Digraph.

In [7]:
nx.number_of_nodes(oxg), nx.number_of_edges(oxg)

(9880, 14907)

In [8]:
adjacency_list = nx.to_dict_of_lists(oxg)

Calculating angles (in degrees) using the law of cosines.

In [9]:
triplets = dict()
angles_list = list()
angles_dict = dict()

for negotiator in adjacency_list.keys():
    angles_dict[negotiator] = dict()
    
    for source, target in itertools.combinations(adjacency_list[negotiator], 2):
        src_lat, src_lon = list(oxg.node[source].values())[0:2]
        tgt_lat, tgt_lon = list(oxg.node[target].values())[0:2]
        neg_lat, neg_lon = list(oxg.node[negotiator].values())[0:2]
        
        # a: from source to negotiator
        d_sn = ox.utils.great_circle_vec(src_lat, src_lon, neg_lat, neg_lon)
        # b: from negotiator to target
        d_nt = ox.utils.great_circle_vec(neg_lat, neg_lon, tgt_lat, tgt_lon)
        # c: from source to target
        d_st = ox.utils.great_circle_vec(src_lat, src_lon, tgt_lat, tgt_lon)
        
        # cos(A) = (c² + b² - a²) / (2bc)
        A = math.acos(((d_nt ** 2.0) + (d_st ** 2.0) - (d_sn ** 2.0)) / (2.0 * d_nt * d_st)) * (180/math.pi)
        # cos(B) = (a² + c² - b²) / (2ac)
        B = math.acos(((d_st ** 2.0) + (d_sn ** 2.0) - (d_nt ** 2.0)) / (2.0 * d_st * d_sn)) * (180/math.pi)
        # cos(C) = (a² + b² - c²) / (2ab)
        C = math.acos(((d_sn ** 2.0) + (d_nt ** 2.0) - (d_st ** 2.0)) / (2.0 * d_sn * d_nt)) * (180/math.pi)
                        
        angles_list.append(C)
        triplets[(source, negotiator, target)] = C
        angles_dict[negotiator][(source, target)] = C # no rounding here, otherwise angles might be zero

Range of streets' convex angles.

In [10]:
min(triplets.values()), max(triplets.values())

(0.001087899271929076, 179.99999743867906)

The number of continuities is the maximum number of pairs of streets connected to a node.

In [11]:
continuity = max([len(list(itertools.combinations(adjacency_list[n], 2))) for n in adjacency_list.keys()])

Defining the interval of each continuity.

In [12]:
increment = max(triplets.values())/continuity

Assigning continuity labels to each triplet of nodes (pairs of streets).

In [13]:
for key in triplets.keys():
    triplet_degree = triplets[key]
    for i in range(1, continuity + 1):
        if (triplet_degree >= math.ceil((i - 1) * increment)) and (triplet_degree < math.ceil(i * increment)):
            triplets[key] = i